# Mimic of the DES night summary for Rubin Observatory for {{ params.day_obs }} of baseline simulation {{ params.sim_version }}

In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2025-05-01"
sim_version = "3.4"

This notebook is a mock-up of a night report for Rubin Observatory/LSST, modeled on the DES night summaries generated by the DES `nightsum` tool.

It is primarily intended as a tool for test and exploration night reporting infrastructure, particularly the use of [Times Square](https://usdf-rsp-dev.slac.stanford.edu/times-square) in conjunction with other tools such as [`schedview`](https://schedview.lsst.io/).

Interested users may check the notebook out of github [here](https://github.com/lsst/schedview_notebooks), install its [dependencies](https://schedview.lsst.io/installation.html), and produce the report with more flexibility (but less convenience) than offered by Times Square. For example, Times Square only supports running data from the recent baseline opsim simulations, while it should be straightforward to run the notebook using bespoke simulations.

In [ ]:
# Validate the input
import re
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', day_obs) is not None
assert re.match(r'^\d+\.\d+$', sim_version) is not None

In [ ]:
print(f"Input day_obs: {day_obs}, input sim_version: {sim_version}")

In [ ]:
import datetime
import os
import sys
from IPython.display import display, HTML
import pandas as pd
import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
import bokeh.transform
import bokeh.layouts
import sqlite3
import numpy as np
import healpy
import astropy
import colorcet
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy
import healpy as hp
import astropy.units as u
from astropy.visualization import ZScaleInterval
from astropy.time import Time
from astropy.coordinates import SkyCoord, get_body
from lsst.resources import ResourcePath

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/user/neilsen/data/rubin_sim_data'

In [ ]:
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages')

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/rubin_scheduler')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/rubin_sim')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/schedview')

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
import rubin_scheduler.site_models
import schedview.compute.astro
import schedview.compute.visits
import uranography
import schedview.plot.visitmap

from rubin_sim import maf
from rubin_scheduler.data import get_baseline
from schedview.compute.camera import LsstCameraFootprintPerimeter
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from uranography.api import Planisphere, make_zscale_linear_cmap
import schedview.plot.survey_skyproj

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

In [ ]:
visit_db_fname = f'/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db'
baseline_opsim_rp = ResourcePath(visit_db_fname)
day_obs_mjd = int(Time(day_obs).mjd)
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"
use_matplotlib = True

In [ ]:
stackers = [
    maf.HourAngleStacker(),
    maf.stackers.ObservationStartDatetime64Stacker(),
    maf.stackers.TeffStacker(),
    maf.stackers.OverheadStacker()
]
try:
    visits = schedview.collect.read_opsim(baseline_opsim_rp, constraint=f"FLOOR(observationStartMJD-0.5)={day_obs_mjd}", stackers=stackers)
except UserWarning:
    print("No visits this night.")
    # Use an empty DataFrame of the right types.
    visits = pd.DataFrame(rubin_scheduler.scheduler.utils.empty_observation()).drop(index=0)

if len(visits):
    visits = schedview.compute.visits.add_coords_tuple(visits)
    visits['previous_filter'] = visits['filter'].shift(1)

## Introduction

This report produces plots analogous to the DES `nightsum` night summaries, but using Rubin Observatory / LSST baseline simulations for its input data.

In this report, the LSST DDF fields are taken to be analogous to the DES SN fields, while all other fields are taken to be analogous to the DES wide survey fields.

## Sun and Moon

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

Modified Julian Date (MJD) is in units of days (UTC).

Local Sidereal Time (LST) is in units of degrees.

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='coordinate')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

RA, dec, alt, and az are all in units of degrees.

## Conditions and statistics

### Numbers of exposures, and gaps between them

In [ ]:
if len(visits):
    overhead_summary = schedview.compute.visits.compute_overhead_summary(visits, night_events.loc['sun_n12_setting','MJD'], night_events.loc['sun_n12_rising','MJD'])
    summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
    display(HTML(summary_table))
else:
    print("No visits")

## Histogram of gaps between exposures

In [ ]:
if len(visits):
    p1 = schedview.plot.create_overhead_histogram(visits)
    p2 = schedview.plot.plot_overhead_vs_slew_distance(visits)
    overhead_plots = bokeh.layouts.row([p1, p2])
    bokeh.io.show(overhead_plots)
else:
    print("No visits")

## Long gaps between exposures

In [ ]:
if len(visits):
    num_gaps = 10
    long_gap_visits = visits.sort_values('overhead', ascending=False).query('overhead>30').loc[:, ['start_date', 'overhead', 'slewDistance', 'filter', 'previous_filter']].sort_values('observationId')
    display(long_gap_visits)
else:
    print("No visits")

## PSF Width

In [ ]:
if len(visits):
    p = schedview.plot.plot_visit_param_vs_time(visits, 'seeingFwhmEff')
    p.yaxis.axis_label = "Effective PSF FWHM (asec)"
    bokeh.io.show(p)
else:
    print("No visits")

Here, the "effective PSF FWHM" serves as a proxy for the measured PSF width in the DES `nightsum`, because it includes an estimates of all modeled contributions.

In an actual report using real data, the width of the PSF measured from the image should be plotted as well, so that differences can be easily noted.

## Instrumental contributions to the PSF

In [ ]:
if len(visits):
    visits = schedview.compute.visits.add_instrumental_fwhm(visits)
    p = schedview.plot.plot_visit_param_vs_time(visits, 'inst_fwhm')
    p.yaxis.axis_label = "Instrumental contribution to the FWHM (asec)"
    bokeh.io.show(p)
else:
    print("No visits")

This perplexes me; I expected the instrumental contribution in simulations to be constant.

## PSF ellipticity

No ellipticity is simulated by opsim.

## Effective exposure time

In [ ]:
if len(visits):
    p = schedview.plot.plot_visit_param_vs_time(visits, 't_eff')
    p.yaxis.axis_label = 'Effecive exposure time (sec.)'
    p.title = "Effective exposure time"
    bokeh.io.show(p)
else:
    print("No visits")

The effective exposure time, t<sub>eff</sub>, is a measure of the depth of an image, mapping monotonically to the (5-sigma point source) limiting magnitude:

m<sub>lim</sub> = m<sub>reference</sub> + 1.25 log10(t<sub>eff</sub>)

where m<sub>reference</sub> is defined such that m<sub>lim</sub> is the 5-sigma point source limiting magnitude for an exposure taken under superb conditions and t<sub>eff</sub> is the actual exposure time. As such, t<sub>eff</sub> is in units of seconds, and expected to be approximately equal to the actual exposure time for exposures taken under superb conditions.

## Sky brightness

In [ ]:
if len(visits):
    p = schedview.plot.plot_visit_param_vs_time(visits, 'skyBrightness')
    p.yaxis.axis_label = 'Sky brightness (mag/asec^2'
    p.title = "Sky brightness"
    bokeh.io.show(p)
else:
    print("No visits")

## Cloud cover

In [ ]:
if len(visits):
    p = schedview.plot.plot_visit_param_vs_time(visits, 'cloud')
    p.yaxis.axis_label = 'Cloud cover'
    p.title = "Cloud cover"
    bokeh.io.show(p)
else:
    print("No visits")

When run with current opsim simulations, all simulations are either completely spoiled (infinite extinction) or clear (no extinction), and what is recorded is a fraction of the sky covered by clouds.

So, where the DES nightsum plots the extinction, what is plotted here is the recorded fraction cloud cover.

## Visit map

In [ ]:
if len(visits):
    vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
        visits=visits,
        night_date=day_obs_date,
        timezone=timezone,
        observatory=observatory,
    )
    bokeh.io.show(vmap)
else:
    print("No visits")

The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The "Armillary sphere" shows the sphere in orthographic projection, with the center point of the projection controlled by the "center alt" and "center az" sliders beneath the plot. A static orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The "Planisphere" shows the sky in [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole, with R.A. increasing counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan. The fill colors are set according to the [descolors palette](https://github.com/DarkEnergySurvey/descolors):

 - <span style='background-color:#56b4e9'>&nbsp;&nbsp;&nbsp;</span><span style='color:#56b4e9'> blue</span>: u band
 - <span style='background-color:#008060'>&nbsp;&nbsp;&nbsp;</span><span style='color:#008060'> green</span>: g band
 - <span style='background-color:#ff4000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ff4000'> red</span>: r band
 - <span style='background-color:#850000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#850000'> brown/crimson</span>: i band
 - <span style='background-color:#6600cc'>&nbsp;&nbsp;&nbsp;</span><span style='color:#6600cc'> purple</span>: z band
 - <span style='background-color:#000000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#000000'> black</span>: y band

Both plots have the following additional annotations:
 - The gray background shows the planned final depth of the LSST survey.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) , completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.

## Survey Progress

### Map depth accumulated so far

In [ ]:
observatory.mjd = night_events.loc['night_middle', 'MJD']
conditions = observatory.return_conditions()

try:
    previous_visits = schedview.collect.read_opsim(visit_db_fname, constraint=f"observationStartMjd < {night_events.loc['sunset', 'MJD']}")
except UserWarning:
    # Use an empty DataFrame of the right types.
    previous_visits = pd.DataFrame(rubin_scheduler.scheduler.utils.empty_observation()).drop(index=0)

In [ ]:
metric = maf.TeffMetric()
nside = 32
if len(previous_visits):
    teff_hpix = schedview.compute.maf.compute_hpix_metric_in_bands(previous_visits, metric, nside=nside)
else:
    teff_hpix = np.zeros(hp.nside2npix(nside))

In [ ]:
if len(visits):
    if use_matplotlib:
        fig = schedview.plot.survey_skyproj.create_hpix_visit_map_grid(visits, teff_hpix, model_observatory, night_events)
    else:
        map_grid = schedview.plot.create_hpix_visit_map_grid(teff_hpix, visits, conditions)
        bokeh.io.show(map_grid)
else:
    print("No visits")

Each subplot represents the progress in the survey in a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

Annotations are similar to those of the "Visit Map" above:

 - Black outlines show the camera footprint of each exposure taken on this night.
 - The blue backdrop represents the accumulated effective exposure time, t<sub>eff</sub> (as defined above) at the start of the night.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.
 - Broken red ovals show a zenith distince of 70° (airmass=2.9) at morning and evening 12° twilight.
   - The dashed portions of each red oval show the extent of what is observable at all during the night.
   - As time progress through the night, the ZD circle maintains its shape, moving counter-clockwise around the pole over the course of the night.

### Map the most recent visit so far

In [ ]:
metric = maf.MaxMetric('observationStartMJD')
if len(previous_visits):
    latest_mjd_hpix = schedview.compute.maf.compute_hpix_metric_in_bands(previous_visits, metric, nside=32)
    time_since_latest_hpix = {b: night_events.loc['sunset', 'MJD'] - latest_mjd_hpix[b].filled(np.nan) for b in latest_mjd_hpix}
else:
    time_since_latest_hpix = np.full(np.inf, hp.nside2npix(nside))

In [ ]:
long_limit = 30
for band in time_since_latest_hpix:
    long_hpix = time_since_latest_hpix[band] > long_limit
    time_since_latest_hpix[band][long_hpix] = np.nan

In [ ]:
if len(visits):
    if use_matplotlib:
        cmap = colorcet.cm.blues_r
        fig = schedview.plot.survey_skyproj.create_hpix_visit_map_grid(visits, time_since_latest_hpix, model_observatory, night_events, vmin=0, vmax=10, cmap=cmap)
    else:
        map_grid = schedview.plot.create_hpix_visit_map_grid(
            time_since_latest_hpix,
            visits,
            conditions,
            scale_limits=[10, 0],
        )
        bokeh.io.show(map_grid)
else:
    print("No visits")

Each subplot represents the time since the most recent visits to each point in the sky, by filter, presented in a Lambert Azimuthal Equal Area Projection, centered at the south celestial pole.

Parts of the sky visited most recently are dark blue, growing fainter and grayer for areas visited less recently. Regions not visited in the last 10 days are not colored.

Other features and annotations are the same as those of the depth map, above.

## DDF Cadence

In [ ]:
time_window_duration = 90

In [ ]:
# offset by 0.5 to get to the right rollover for day_obs, and to make the range inclusive
ddf_start_time = Time(day_obs_mjd - time_window_duration - 0.5, format='mjd')
ddf_end_time = Time(day_obs_mjd + 0.5, format='mjd')

stackers = [
    maf.stackers.ObservationStartDatetime64Stacker(),
    maf.stackers.TeffStacker(),
    maf.stackers.DayObsISOStacker(),
]

try:
    ddf_visits = schedview.collect.opsim.read_ddf_visits(visit_db_fname, ddf_start_time, ddf_end_time, stackers=stackers)
except UserWarning:
    # Use an empty DataFrame of the right types.
    ddf_visits = pd.DataFrame(rubin_scheduler.scheduler.utils.empty_observation()).drop(index=0)

In [ ]:
if len(ddf_visits):
    nightly_ddf = schedview.compute.visits.accum_teff_by_night(ddf_visits)
    cadence_plots = schedview.plot.create_cadence_plot(nightly_ddf, day_obs_mjd - time_window_duration, day_obs_mjd)
    bokeh.io.show(cadence_plots)
else:
    print("No DDF visits")

The y-axis (height of the vertical bars) represents the accumulated effective exposure time, t<sub>eff</sub> (as defined above) accumulated over all exposures on the field for the night, colored by filter.

## Table of exposures

In [ ]:
if len(visits):
    displayed_columns = ['start_date', 'fieldRA', 'fieldDec', 'filter', 'visitExposureTime', 'numExposures', 't_eff', 'skyBrightness', 'seeingFwhmEff', 'cloud', 'note']
    displayed_visits_df = visits.loc[:, displayed_columns]
    with pd.option_context('display.max_rows', 2000):
        display(displayed_visits_df)
else:
    print("No visits")